# Prompt Templates
**Prompt Templates** are tools designed to generate structured prompts for a language model. They take user input and parameters as input and use these to create prompts that guide the model's response.

Prompt Templates help to ensure that the model’s output is relevant, contextually aware, and coherent.

## Types of Prompt Templates
1. **String PromptTemplates**
These prompt templates are used to format a single string, and generally are used for simpler inputs.
2. **Chat Prompt Templates**:
These templates are tailored for chat-based models and can include multiple messages, user roles, and system instructions.
3. **MessagesPlaceholder**
This prompt template is responsible for adding a list of messages in a particular place. In the above ChatPromptTemplate, we saw how we could format two messages, each one a string. But what if we wanted the user to pass in a list of messages that we would slot into a particular spot? This is how you use MessagesPlaceholder.


## Setup

In [ ]:
%pip install langchain_community langchain langchain-nvidia-ai-endpoints

In [3]:
import getpass
import os

# Set up your api key
if not os.environ.get("NVIDIA_API_KEY"):
  os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA: ")

Enter API key for NVIDIA: ··········


In [4]:
from langchain.chat_models import init_chat_model
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

# Select the chat model
llm = init_chat_model('meta/llama-3.1-70b-instruct', model_provider = 'nvidia')

# Select the embedding model
embeddings = NVIDIAEmbeddings(model='NV-Embed-QA')

## 1. String PromptTemplates

In [9]:
from langchain_core.prompts import PromptTemplate

template = """Tell me about {topic}"""

prompt_template = PromptTemplate.from_template(template)

template


'Tell me about {topic}'

In [11]:
# Assign topic to prompt
prompt_template = prompt_template.format(topic="Dinousors")
prompt_template

'Tell me about Dinousors'

In [15]:
from langchain.chat_models import init_chat_model
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_core.prompts import PromptTemplate


prompt_template = PromptTemplate.from_template(template)

# Use prompt_template in the chain
chain = prompt_template | llm

# Invoke the chain with the input variables
chain.invoke({"topic": "Dinousors"}) # Pass 'Dinousors' as input variable

AIMessage(content="Dinosaurs! Those magnificent, mighty, and mythical creatures of the ancient world. Dinosaurs were a diverse group of reptiles that roamed the Earth during the Mesozoic Era, which lasted from about 252 million to 66 million years ago. Here's a brief overview:\n\n**Types of Dinosaurs:**\n\n1. **Sauropods** (e.g., Brachiosaurus, Diplodocus): Long-necked, long-tailed herbivores.\n2. **Theropods** (e.g., Tyrannosaurus, Velociraptor): Meat-eating predators with hollow bones and three-toed limbs.\n3. **Ornithischians** (e.g., Triceratops, Stegosaurus): Armored, plant-eating dinosaurs with distinct pelvis structures.\n4. **Ceratopsians** (e.g., Psittacosaurus, Styracosaurus): Horned, herbivorous dinosaurs with distinctive frills and horns.\n\n**Interesting Facts:**\n\n1. **First Dinosaur Discovery**: The first dinosaur fossil was found by William Buckland in 1824 (Megalosaurus bucklandi).\n2. **Largest Land Animal**: Argentinosaurus, a sauropod, is currently the largest know

## 2. Chat PromptTemplte
- ChatPromptTemplate can be used to include a conversation history as a prompt.

- Messages are structured as tuples in the format (role , message ) and are created as a list.

- These prompt templates are used to format a list of messages. These "templates" consist of a list of templates themselves.

In [18]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate([
    ('system', "You are a helpful assistance"),
    ('user', 'Tell me about {topic}')
])

chat_prompt.format(topic="Germany")

'System: You are a helpful assistance\nHuman: Tell me about Germany'

In [19]:

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a friendly AI assistant. Your name is {name}."),
        ("human", "Hello how are you!"),
        ("ai", "Hello! How can I assist you?"),
        ("human", "{user_input}"),
    ]
)

# Create chat messages
messages = chat_template.format_messages(name="beru", user_input="What is your name?")
messages


[SystemMessage(content='You are a friendly AI assistant. Your name is beru.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello how are you!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I assist you?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]

In [21]:
llm.invoke(messages).content

"My name is Beru, nice to meet you! I'm a friendly AI assistant, here to help and chat with you. How about you, how's your day going so far?"

## 3. MessagePlaceHolder
- LangChain also provides a MessagePlaceholder , which provides complete control over rendering messages during formatting.
- This can be useful if you’re unsure which roles to use in a message prompt template or if you want to insert a list of messages during formatting.
- This prompt template is responsible for adding a list of messages in a particular place. In the above ChatPromptTemplate, we saw how we could format two messages, each one a string. But what if we wanted the user to pass in a list of messages that we would slot into a particular spot

In [25]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant you have to assist on any given topic as best as you can"
    ),
    MessagesPlaceholder(variable_name='Assistance'),
    ('human', 'how to book a flight ticket')
])

prompt_template

ChatPromptTemplate(input_variables=['Assistance'], input_types={'Assistance': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core

In [28]:
formatted_prompt = prompt_template.format_prompt(
    Assistance=[
        ('human', 'hello! I am beru. how are you'),
        ('ai', 'Nice to meet you! I am good')
    ])

print(formatted_prompt)

messages=[SystemMessage(content='You are a helpful assistant you have to assist on any given topic as best as you can', additional_kwargs={}, response_metadata={}), HumanMessage(content='hello! I am beru. how are you', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you! I am good', additional_kwargs={}, response_metadata={}), HumanMessage(content='how to book a flight ticket', additional_kwargs={}, response_metadata={})]


In [31]:
from langchain_core.output_parsers import StrOutputParser
chain = chat_prompt | llm | StrOutputParser()


result = chain.invoke(formatted_prompt.to_messages())

In [32]:
from IPython.display import display, Markdown

display(Markdown(result))

You want to know how to book a flight ticket. Here's a step-by-step guide to help you book a flight ticket:

**Booking a Flight Ticket: A Step-by-Step Guide**

**Option 1: Book through an Airline's Website**

1. Go to the website of the airline you want to fly with (e.g., [www.airindia.in](http://www.airindia.in), [www.indiGo.in](http://www.indiGo.in), etc.).
2. Click on the "Book a Flight" or "Plan a Trip" tab.
3. Enter your travel details:
	* From (departure city)
	* To (arrival city)
	* Departure Date
	* Return Date (if applicable)
	* Number of Passengers
4. Choose your flight:
	* Select your preferred flight from the search results.
	* Check the flight schedule, fare, and availability.
5. Enter passenger details:
	* Name
	* Age
	* Contact Information
	* Email ID
6. Make the payment:
	* Pay using a credit/debit card or online banking.
	* Receive an email confirmation with your ticket details.

**Option 2: Book through a Travel Portal**

1. Visit a travel portal website (e.g., [www.makemytrip.com](http://www.makemytrip.com), [www.expedia.com](http://www.expedia.com), etc.).
2. Enter your travel details:
	* From (departure city)
	* To (arrival city)
	* Departure Date
	* Return Date (if applicable)
	* Number of Passengers
3. Compare prices and choose your flight:
	* Browse through the search results to find the best deals.
	* Filter flights by price, duration, and airline.
4. Enter passenger details:
	* Name
	* Age